# About this notebook...
###Documention prepared by **Jesus Perez Colino**.Version 0.4, Released 01/04/2013, Beta


- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: This notebook was made as a documentation to support a course that I gave in Germany 2013, about Scientific Python and Quantitative Finance. This is a Python implementation of the well-know **Least-Square Monte Carlo (LSMC) method** for the valuation of **American Options** as it appears in the **Longstaff-Schwartz (2001)** paper. 


- **Reference**: [Longstaff-Schwartz (2001): "Valuing American Options by Simulation: A Simple Least-Squares Approach." Review of Financial Studies, Vol. 14, 113-147](https://www.google.de/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0CCEQFjAAahUKEwiXtNSZm4rHAhXHOhQKHTjBD3k&url=https%3A%2F%2Fpeople.math.ethz.ch%2F~hjfurrer%2Fteaching%2FLongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf&ei=7PO9VZeOBcf1ULiCv8gH&usg=AFQjCNFQr1r_Cf_pxylg_amU3TFOZVDc8w&sig2=ixZnX_wWQ48G66BMuQTPZA&bvm=bv.99261572,d.d24)


- **Python & packages versions** to reproduce the results of this notebook: 

In [1]:
import IPython
import numpy as np
from sys import version 
print '='*85
print 'Python version:     ' + version
print 'Numpy version:      ' + np.__version__
print 'IPython version:    ' + IPython.__version__
print '='*85

Python version:     2.7.10 |Anaconda 2.3.0 (x86_64)| (default, May 28 2015, 17:04:42) 
[GCC 4.2.1 (Apple Inc. build 5577)]
Numpy version:      1.9.2
IPython version:    3.2.0


#A Python Class for American Options pricing using LSMC - Longstaff-Schwartz (2001)

In [206]:
import numpy as np

class AmericanOptionsLSMC(object):
    """ Class for American options pricing using Longstaff-Schwartz (2001):
    "Valuing American Options by Simulation: A Simple Least-Squares Approach."
    Review of Financial Studies, Vol. 14, 113-147.
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    
    Unitest(doctest): 
    >>> AmericanPUT = AmericanOptionsLSMC('put', 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
    >>> AmericanPUT.price
    4.4731177017712209
    """

    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            self.T = float(T)
            self.M = int(M)
            self.r = float(r)
            self.div = float(div)
            self.sigma = float(sigma)
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Options parameters')

        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / self.M
        self.discount = np.exp(-self.r * self.time_unit)

    @property
    def MCprice_matrix(self, seed = 123):
        """ Returns MC price matrix rows: time columns: price-path simulation """
        np.random.seed(seed)
        MCprice_matrix = np.zeros((self.M + 1, self.simulations),dtype=np.float64)
        MCprice_matrix[0,:] = self.S0
        for t in xrange(1, self.M + 1):
            ran = np.random.standard_normal( self.simulations / 2)
            ran = np.concatenate((ran, -ran))
            MCprice_matrix[t, :] = (MCprice_matrix[t - 1, :]
                                  * np.exp((self.r - self.sigma ** 2 / 2) * self.time_unit
                                  + self.sigma * ran * np.sqrt(self.time_unit)))
        return MCprice_matrix

    @property
    def MCpayoff(self):
        """Returns the inner-value of American Option"""
        if self.option_type == 'call':
            payoff = np.maximum(self.MCprice_matrix - self.strike,
                           np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        else:
            payoff = np.maximum(self.strike - self.MCprice_matrix,
                            np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        return payoff

    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        for t in range(self.M - 1, 0 , -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 5)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)

        return value_matrix[1,:] * self.discount


    @property
    def price(self): return np.sum(self.value_vector) / self.simulations

In [207]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)

In [208]:
AmericanPUT = AmericanOptionsLSMC('put', 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
AmericanPUT.price

4.4731177017712209

In [204]:
def prices():
    for S0 in (36., 38., 40., 42., 44.):  # initial stock price values
        for vol in (0.2, 0.4):  # volatility values
            for T in (1.0, 2.0):  # times-to-maturity
                AmericanPUT = AmericanOptionsLSMC('put', S0, 40., T, 50, 0.06, 0.06, vol, 1500)
                print "Initial price: %4.1f, Sigma: %4.2f, Expire: %2.1f --> Option Value %8.3f" % (S0, vol, T, AmericanPUT.price)

In [205]:
from time import time
t0 = time()
optionValues = prices()  # calculate all values
t1 = time(); d1 = t1 - t0
print "Duration in Seconds %6.3f" % d1

Initial price: 36.0, Sigma: 0.20, Expire: 1.0 --> Option Value    4.439
Initial price: 36.0, Sigma: 0.20, Expire: 2.0 --> Option Value    4.779
Initial price: 36.0, Sigma: 0.40, Expire: 1.0 --> Option Value    7.135
Initial price: 36.0, Sigma: 0.40, Expire: 2.0 --> Option Value    8.459
Initial price: 38.0, Sigma: 0.20, Expire: 1.0 --> Option Value    3.225
Initial price: 38.0, Sigma: 0.20, Expire: 2.0 --> Option Value    3.726
Initial price: 38.0, Sigma: 0.40, Expire: 1.0 --> Option Value    6.134
Initial price: 38.0, Sigma: 0.40, Expire: 2.0 --> Option Value    7.666
Initial price: 40.0, Sigma: 0.20, Expire: 1.0 --> Option Value    2.296
Initial price: 40.0, Sigma: 0.20, Expire: 2.0 --> Option Value    2.808
Initial price: 40.0, Sigma: 0.40, Expire: 1.0 --> Option Value    5.201
Initial price: 40.0, Sigma: 0.40, Expire: 2.0 --> Option Value    6.815
Initial price: 42.0, Sigma: 0.20, Expire: 1.0 --> Option Value    1.589
Initial price: 42.0, Sigma: 0.20, Expire: 2.0 --> Option Value  